In [ ]:
# Define the model
model = models.Sequential([
    layers.Input(shape=(1024,)),  # Input layer for 1024-dimensional embeddings
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(mlb.classes_), activation='sigmoid')  # Output layer for multi-label classification
])

# Compile the model with binary cross-entropy loss for multi-label classification
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC(name="AUC", multi_label=True)])

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64)


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers

# Custom F2 loss function
def f2_loss(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    tp = tf.reduce_sum(y_true * y_pred, axis=0)
    fp = tf.reduce_sum((1 - y_true) * y_pred, axis=0)
    fn = tf.reduce_sum(y_true * (1 - y_pred), axis=0)
    f2 = (5 * tp) / (5 * tp + 4 * fn + fp + 1e-8)
    return 1 - tf.reduce_mean(f2)  # 1 - F2 to minimize loss

# Model architecture with increased complexity
def create_model():
    model = models.Sequential([
        layers.Input(shape=(1024,)),
        layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.5),

        layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.5),

        layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),

        layers.Dense(1400, activation='sigmoid')  # Multi-label output
    ])
    model.compile(optimizer='adam', loss=f2_loss, metrics=['binary_accuracy'])
    return model

# Create the model
# model = create_model()

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=64)
